<a href="https://colab.research.google.com/github/MarylouBer/MLD_Data_Engineering/blob/main/NZDemographics_SQLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Code to avoid prettytable default error.
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'


In [ ]:
#Install Pandas
!pip install pandas

In [ ]:
# Creating a connection to your SQLite database (con)
import sqlite3
con = sqlite3.connect("NZDemographics.db")
cur = con.cursor()

In [ ]:
# uploading files from my local maching to google drive
# those files where downloaded from the following website: https://www.stats.govt.nz/large-datasets/csv-files-for-download/
from google.colab import files
uploaded = files.upload()

# load the CSV into a DataFrame
import pandas as pd
df1 = pd.read_csv('DemographicsByEthnicity.csv')
df2 = pd.read_csv('Ethnic.csv')
df3 = pd.read_csv('Area.csv')
df4 = pd.read_csv('Sex.csv')
df5 = pd.read_csv('Age.csv')
df6 = pd.read_csv('Year.csv')

In [ ]:
# Insert DataFrames into SQL tables
df1.to_sql("DemographicsByEthnicity", con, if_exists='replace', index=False, method="multi", chunksize=1000)
df2.to_sql("Ethnic", con, if_exists='replace', index=False, method="multi", chunksize=1000)
df3.to_sql("Area", con, if_exists='replace', index=False, method="multi", chunksize=1000)
df4.to_sql("Sex", con, if_exists='replace', index=False, method="multi", chunksize=1000)
df5.to_sql("Age", con, if_exists='replace', index=False, method="multi", chunksize=1000)
df6.to_sql("Year", con, if_exists='replace', index=False, method="multi", chunksize=1000)

In [ ]:
#Load the SQL Magic module
!pip install ipython-sql
%load_ext sql
%sql sqlite:///NZDemographics.db
%reload_ext sql

In [ ]:
# count number of distinct ethnicity codes in the table Ethnic
%sql SELECT DISTINCT COUNT(Code) FROM Ethnic

In [ ]:
# show the most present gender using a sub-query
%sql SELECT s.description \
     FROM Sex s \
     WHERE s.code = (SELECT Sex FROM DemographicsByEthnicity GROUP BY Sex ORDER BY COUNT(*) DESC LIMIT 1)

In [ ]:
# shows the different tables of my database and their entities

# Step 1: Get the list of tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Step 2: Loop through each table and fetch its entities
for table in tables:
    table_name = table[0]
    print(f"Columns in table {table_name}:")

    # Fetch the entity for the current table
    cur.execute(f"PRAGMA table_info({table_name});")
    columns = cur.fetchall()

    # Display the entity names
    for column in columns:
        print(f"- {column[1]}")  # column[1] contains the column name
    print()  # Blank line between tables

# Close the connection
con.close()